In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render ul li{font-size:22pt; line-height:30px;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

In [2]:
# ============================================================
# [단락 0] 라이브러리
# ============================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [4]:
# ============================================================
# [단락 1] 데이터 로드
# ============================================================
path = r"C:\ai\lecNote\1st_Project\data\서울_일반음식점_전처리_폐업률머지.csv"
df = pd.read_csv(path, low_memory=False)

print(df.shape)
print(df[["구", "업태_그룹", "폐업률_3년이내_pct"]].head())

(241267, 24)
     구   업태_그룹  폐업률_3년이내_pct
0  은평구      기타          31.0
1  은평구      한식          33.2
2  서초구  분식/간편식          39.7
3  서초구   양식/외식          30.0
4   중구   주점/치킨          24.5


In [5]:
# ============================================================
# [단락 2] (구, 업태_그룹) 조합을 유니크하게 만들기
#  - 폐업률_3년이내_pct는 (구,업태) 조합별로 동일하므로
#    조합당 1개 행만 남겨 학습 데이터로 사용
# ============================================================
g = (
    df[["구", "업태_그룹", "폐업률_3년이내_pct"]]
      .dropna()
      .groupby(["구", "업태_그룹"], as_index=False)
      .agg({"폐업률_3년이내_pct": "first"})
)

y = g["폐업률_3년이내_pct"].astype("float32").values

print("유니크 조합 수:", len(g))
print(g.head())


유니크 조합 수: 175
     구   업태_그룹  폐업률_3년이내_pct
0  강남구      기타          38.3
1  강남구  분식/간편식          34.0
2  강남구   양식/외식          30.3
3  강남구   주점/치킨          33.3
4  강남구    중/일식          28.1


In [6]:
# ============================================================
# [단락 3] 라벨(정수) 인코딩
#  - 문자열 범주(구/업태)를 정수 ID로 변환
#  - Embedding 레이어에 넣기 위해 필요
# ============================================================
g["구_id"], gu_vocab = pd.factorize(g["구"])
g["업태_id"], biz_vocab = pd.factorize(g["업태_그룹"])

X_gu = g["구_id"].astype("int32").values
X_biz = g["업태_id"].astype("int32").values

n_gu = len(gu_vocab)
n_biz = len(biz_vocab)

print("구 종류 수:", n_gu, "| 업태_그룹 종류 수:", n_biz)


구 종류 수: 25 | 업태_그룹 종류 수: 7


In [7]:
# ============================================================
# [단락 4] Train / Val / Test split
# ============================================================
idx = np.arange(len(g))
tr, te = train_test_split(idx, test_size=0.2, random_state=42)
tr, va = train_test_split(tr, test_size=0.2, random_state=42)

X_tr = {"gu": X_gu[tr], "biz": X_biz[tr]}
X_va = {"gu": X_gu[va], "biz": X_biz[va]}
X_te = {"gu": X_gu[te], "biz": X_biz[te]}

y_tr, y_va, y_te = y[tr], y[va], y[te]

print("train/val/test size:", len(tr), len(va), len(te))


train/val/test size: 112 28 35


In [8]:
# ============================================================
# [단락 5] 모델 정의 (Embedding + Dense) : 회귀
#  - 입력: 구_id, 업태_id (정수)
#  - 출력: 폐업률_3년이내_pct (연속값) 1개
# ============================================================
def emb_dim(vocab_size: int) -> int:
    # 작은 범주 수에서는 너무 큰 임베딩은 과적합이 쉬움 → 적당히 제한
    return int(min(16, max(4, round(vocab_size ** 0.5))))

gu_in  = keras.Input(shape=(), dtype=tf.int32, name="gu")
biz_in = keras.Input(shape=(), dtype=tf.int32, name="biz")

gu_emb  = layers.Embedding(input_dim=n_gu + 1,  output_dim=emb_dim(n_gu + 1))(gu_in)
biz_emb = layers.Embedding(input_dim=n_biz + 1, output_dim=emb_dim(n_biz + 1))(biz_in)

x = layers.Concatenate()([layers.Flatten()(gu_emb), layers.Flatten()(biz_emb)])
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(16, activation="relu")(x)

out = layers.Dense(1, activation="linear")(x)

model = keras.Model(inputs=[gu_in, biz_in], outputs=out)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="mse",
    metrics=[keras.metrics.MAE]
)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 gu (InputLayer)                [(None,)]            0           []                               
                                                                                                  
 biz (InputLayer)               [(None,)]            0           []                               
                                                                                                  
 embedding (Embedding)          (None, 5)            130         ['gu[0][0]']                     
                                                                                                  
 embedding_1 (Embedding)        (None, 4)            32          ['biz[0][0]']                    
                                                                                              

In [9]:
# ============================================================
# [단락 6] 학습 (가중치 없음)
# ============================================================
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)
]

history = model.fit(
    X_tr, y_tr,
    validation_data=(X_va, y_va),
    epochs=300,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/300
4/4 [==============================] - 1s 46ms/step - loss: 1018.1011 - mean_absolute_error: 31.4153 - val_loss: 978.3790 - val_mean_absolute_error: 30.7799
Epoch 2/300
4/4 [==============================] - 0s 12ms/step - loss: 1016.1058 - mean_absolute_error: 31.3835 - val_loss: 976.4371 - val_mean_absolute_error: 30.7482
Epoch 3/300
4/4 [==============================] - 0s 10ms/step - loss: 1013.9632 - mean_absolute_error: 31.3490 - val_loss: 974.3907 - val_mean_absolute_error: 30.7147
Epoch 4/300
4/4 [==============================] - 0s 10ms/step - loss: 1011.6592 - mean_absolute_error: 31.3124 - val_loss: 972.1196 - val_mean_absolute_error: 30.6776
Epoch 5/300
4/4 [==============================] - 0s 6ms/step - loss: 1009.1956 - mean_absolute_error: 31.2726 - val_loss: 969.5905 - val_mean_absolute_error: 30.6363
Epoch 6/300
4/4 [==============================] - 0s 11ms/step - loss: 1006.2538 - mean_absolute_error: 31.2244 - val_loss: 966.7128 - val_mean_absolute_er

Epoch 50/300
4/4 [==============================] - 0s 11ms/step - loss: 42.3367 - mean_absolute_error: 5.0982 - val_loss: 26.1424 - val_mean_absolute_error: 3.4975
Epoch 51/300
4/4 [==============================] - 0s 11ms/step - loss: 42.5784 - mean_absolute_error: 5.0967 - val_loss: 25.4574 - val_mean_absolute_error: 3.5019
Epoch 52/300
4/4 [==============================] - 0s 9ms/step - loss: 38.6595 - mean_absolute_error: 4.7316 - val_loss: 24.9491 - val_mean_absolute_error: 3.4877
Epoch 53/300
4/4 [==============================] - 0s 9ms/step - loss: 37.5852 - mean_absolute_error: 4.7595 - val_loss: 24.4625 - val_mean_absolute_error: 3.4619
Epoch 54/300
4/4 [==============================] - 0s 5ms/step - loss: 36.3119 - mean_absolute_error: 4.5759 - val_loss: 24.0136 - val_mean_absolute_error: 3.4477
Epoch 55/300
4/4 [==============================] - 0s 8ms/step - loss: 40.6055 - mean_absolute_error: 5.1278 - val_loss: 23.6689 - val_mean_absolute_error: 3.4373
Epoch 56/300
4

Epoch 100/300
4/4 [==============================] - 0s 9ms/step - loss: 13.9516 - mean_absolute_error: 2.9141 - val_loss: 14.9966 - val_mean_absolute_error: 3.3033
Epoch 101/300
4/4 [==============================] - 0s 6ms/step - loss: 21.2436 - mean_absolute_error: 3.4952 - val_loss: 14.9500 - val_mean_absolute_error: 3.2979
Epoch 102/300
4/4 [==============================] - 0s 7ms/step - loss: 18.2192 - mean_absolute_error: 3.4934 - val_loss: 14.9259 - val_mean_absolute_error: 3.2902
Epoch 103/300
4/4 [==============================] - 0s 12ms/step - loss: 17.7925 - mean_absolute_error: 3.2370 - val_loss: 14.8803 - val_mean_absolute_error: 3.2889
Epoch 104/300
4/4 [==============================] - 0s 10ms/step - loss: 17.8841 - mean_absolute_error: 3.4975 - val_loss: 14.9412 - val_mean_absolute_error: 3.2990
Epoch 105/300
4/4 [==============================] - 0s 7ms/step - loss: 19.0411 - mean_absolute_error: 3.4709 - val_loss: 14.9766 - val_mean_absolute_error: 3.3052
Epoch 10

4/4 [==============================] - 0s 12ms/step - loss: 23.4594 - mean_absolute_error: 3.8461 - val_loss: 14.0569 - val_mean_absolute_error: 3.2094
Epoch 150/300
4/4 [==============================] - 0s 9ms/step - loss: 19.8166 - mean_absolute_error: 3.4250 - val_loss: 14.1038 - val_mean_absolute_error: 3.2142
Epoch 151/300
4/4 [==============================] - 0s 11ms/step - loss: 15.0163 - mean_absolute_error: 3.0212 - val_loss: 14.1097 - val_mean_absolute_error: 3.2123


In [10]:
# ============================================================
# [단락 7] 평가 + 예측 일부 출력
# ============================================================
res = model.evaluate(X_te, y_te, verbose=0)
print("\n[Test] MSE, MAE =", res)

pred = model.predict(X_te, verbose=0).reshape(-1)

print("\n예측 vs 실제(10개)")
for p, t in list(zip(pred[:10], y_te[:10])):
    print(f"pred={p:.1f}%  true={t:.1f}%")



[Test] MSE, MAE = [11.092499732971191, 2.7560136318206787]

예측 vs 실제(10개)
pred=19.9%  true=23.2%
pred=22.4%  true=24.6%
pred=26.9%  true=28.6%
pred=42.3%  true=48.0%
pred=32.5%  true=30.6%
pred=31.6%  true=27.9%
pred=35.3%  true=37.2%
pred=38.1%  true=45.6%
pred=36.3%  true=37.3%
pred=32.3%  true=36.5%


In [11]:
# ============================================================
# [단락 8] 검증(Val) 예측 결과 출력 (구/업태 이름 포함, 오차 큰 순)
#  - 소수점 1자리로 "표시" 강제
# ============================================================
pred_va = model.predict(X_va, verbose=0).reshape(-1)

# 검증 지표(소수 1자리 출력)
mae_va  = float(np.mean(np.abs(pred_va - y_va)))
mse_va  = float(np.mean((pred_va - y_va) ** 2))
rmse_va = float(np.sqrt(mse_va))

print(f"\n[Validation] MAE = {mae_va:.1f} | RMSE = {rmse_va:.1f} | MSE = {mse_va:.1f}")

# va 인덱스에 해당하는 (구, 업태_그룹) 가져오기
val_result = g.iloc[va][["구", "업태_그룹"]].copy()
val_result["true_pct"] = y_va
val_result["pred_pct"] = pred_va
val_result["abs_error"] = np.abs(val_result["pred_pct"] - val_result["true_pct"])

# ✅ 출력용: 소수 1자리로 강제 표시(문자열 포맷)
val_result["true_pct"]  = val_result["true_pct"].map(lambda x: f"{x:.1f}")
val_result["pred_pct"]  = val_result["pred_pct"].map(lambda x: f"{x:.1f}")
val_result["abs_error"] = val_result["abs_error"].map(lambda x: f"{x:.1f}")

print("\n[검증 예측 결과]")
print(val_result.to_string(index=False))

print("\n[오차 큰 순]")
print(val_result.sort_values("abs_error", ascending=False).to_string(index=False))



[Validation] MAE = 3.1 | RMSE = 3.7 | MSE = 13.6

[검증 예측 결과]
   구  업태_그룹 true_pct pred_pct abs_error
 도봉구     기타     32.1     34.7       2.6
 관악구     카페     31.7     30.2       1.5
 동작구     카페     29.6     26.1       3.5
 양천구  주점/치킨     34.1     33.6       0.5
 송파구     카페     33.1     29.2       3.9
동대문구     카페     24.6     29.8       5.2
 금천구     기타     32.5     34.9       2.4
영등포구  주점/치킨     31.1     30.0       1.1
 은평구     한식     33.2     32.8       0.4
 금천구     카페     25.7     30.7       5.0
 노원구  양식/외식     29.7     30.4       0.7
 서초구  양식/외식     30.0     27.2       2.8
동대문구  주점/치킨     29.9     31.2       1.3
 광진구 분식/간편식     43.3     37.0       6.3
 송파구  양식/외식     35.0     31.0       4.0
 종로구  주점/치킨     24.3     19.5       4.8
 종로구     카페     19.9     18.1       1.8
서대문구     한식     27.8     31.1       3.3
 서초구     카페     31.2     25.5       5.7
 성동구  양식/외식     27.4     29.2       1.8
  중구     카페     24.0     26.6       2.6
 중랑구 분식/간편식     45.1     38.6       6.5
 은평구     기타     31